In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np 
import pandas as pd
import math

import torch
from torch import nn
import torch.nn.functional as F
from torch.nn import TransformerEncoder, TransformerEncoderLayer
from torch.utils.data import DataLoader, Dataset
from collections import Counter
from torch.nn.utils.rnn import pad_sequence
from torchvision.transforms import ToTensor, Lambda, Compose
import matplotlib.pyplot as plt

from sklearn.model_selection  import train_test_split
from sklearn.model_selection import KFold


from transformers import AutoModel, AutoTokenizer, BertTokenizer, AutoConfig

import itertools
import gc
import os 
import random

import spacy
import time
import timeit

In [ ]:
def seed_everything(seed=1234):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
seed_everything(seed=1326)

# HyperParameters

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# # for TPU
# device = xm.xla_device()
# torch.set_default_tensor_type('torch.FloatTensor')

batch_size = 256
epochs = 4 # The number of epochs


 # load dataframe

In [ ]:

train_path = '../input/seti-breakthrough-listen/train'

train_labels = pd.read_csv('../input/seti-breakthrough-listen/train_labels.csv')
train_labels

In [ ]:
def load_image(filename, data_path=train_path):
    data = np.load(f'{data_path}/{filename[0]}/{filename}.npy').astype(np.float32)
    for i in range(data.shape[0]):
        data[i] -= data[i].mean()
        data[i] /= data[i].std()
    return data


res = load_image(train_labels.iloc[0][0])
res.shape

In [ ]:
class ImageDataset(Dataset):
    def __init__(self, df):
        self.df = df

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        image = load_image(self.df.iloc[idx][0])
        label = self.df.iloc[idx][1]

        return image, label
        
        
train_data = ImageDataset(train_labels)

# Model


In [ ]:
class SETIModel(nn.Module):
    def __init__(self):
        super(SETIModel,self).__init__()
        self.dropout = torch.nn.Dropout(0.5)
        self.LRelu = torch.nn.LeakyReLU(negative_slope=0.01)
        self.Conv1 = torch.nn.Conv2d(6,6,5, stride = 2 )
        self.Conv2 = torch.nn.Conv2d(6,3,5, stride = 2 )
        self.Conv3 = torch.nn.Conv2d(3,3,5, stride = 2 )
        self.linear1 = torch.nn.Linear(2697, 512)
        self.linear2 = torch.nn.Linear(512, 2)


                       
    def forward(self,data):
        x = self.Conv1(data)
        x = self.dropout(x)
        x = self.Conv2(x)
        x = self.dropout(x)
        x = self.Conv3(x)
        x = torch.flatten(x, 1)
        x = self.linear1(x)
        x = self.LRelu(x)
        x = self.dropout(x)
        x = self.linear2(x)
#         print(f'x is {x.size()}' )
        return x




model = SETIModel().to(device)
torch.save(model.state_dict(), 'initialModel')


# Train and Eval

In [ ]:
import time

bptt = 35

criterion = nn.CrossEntropyLoss()
lr = 0.0002 # learning rate
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
# scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1, gamma=0.95, verbose = True)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1, gamma=0.95)


def train(model):
    model.train() # Turn on the train mode
    return_loss = []
    total_loss = 0.
    start_time = time.time()
    batch = 0
    for data, targets in train_loader:
        model.train()
        optimizer.zero_grad()
      
        data = torch.tensor(data).float().to(device)
        final_output = model(data).to(device)
        
        targets = torch.tensor(targets).to(device)
#         print(targets.size())
        loss = criterion(final_output, targets)


        loss.backward()
#         torch.nn.utils.clip_grad_norm_(model.parameters(), 0.5)
        optimizer.step()

        total_loss += loss.item()
        return_loss.append(loss.item())
        log_interval = 100
        batch += 1
#         if batch % log_interval == 0 and batch > 0:
#             cur_loss = total_loss / log_interval
#             elapsed = time.time() - start_time
#             print('| epoch {:3d} | {:5d}/{:5d} batches | '
#                   'lr {:02.2f} | ms/batch {:5.2f} | '
#                   'loss {:5.2f} | ppl {:8.2f}'.format(
#                     epoch, batch, len(train_data) // bptt, scheduler.get_last_lr()[0],
#                     elapsed * 1000 / log_interval,
#                     cur_loss, np.exp(cur_loss)))
        total_loss = 0
        start_time = time.time()
        
    return torch.mean(torch.tensor(return_loss))

def evaluate(eval_model):
#     losses = []
    eval_model.eval() # Turn on the evaluation mode
#     total_loss = 0.
    total_loss = []
    with torch.no_grad():
        for data, targets in val_loader:
            
            data = torch.tensor(data).float().to(device)
            final_output = model(data).to(device)
            targets = torch.tensor(targets).to(device)
            loss = criterion(final_output, targets)
#             loss = criterion(final_output, targets)
            currLoss = criterion(final_output, targets).item()
#             total_loss += len(data) * currLoss
            total_loss.append(currLoss)

#     return total_loss
    return torch.mean(torch.tensor(total_loss))

# KFold Cross validation

In [ ]:
# KFolds

kfold = KFold(n_splits= 5 , shuffle=True)


best_val_loss = float("inf")
best_model = None
epoch_averages = [[] for _ in range(epochs)]


fold_scores = []

print(f'length of dataset is {len(train_labels)}')

for fold, (train_ids, val_ids) in enumerate(kfold.split(train_labels)):
    
    # Print
    print(f'FOLD {fold}')
    print('--------------------------------')

    # reset weights for the new fold
    model.load_state_dict(torch.load('initialModel'))

    
    
    # Sample elements randomly from a given list of ids, no replacement.
    train_subsampler = torch.utils.data.SubsetRandomSampler(train_ids)
    val_subsampler = torch.utils.data.SubsetRandomSampler(val_ids)
    
    # Define data loaders for training and testing data in this fold
    train_loader = DataLoader(train_data, batch_size = batch_size, sampler = train_subsampler, num_workers = 4)
    val_loader = DataLoader(train_data, batch_size = batch_size, sampler = val_subsampler, num_workers = 4)
    
    
    train_losses = []
    val_losses = []
    
    # Run the training loop for defined number of epochs
    for epoch in range(1, epochs +1):
        # Print epoch
        print(f'Starting epoch {epoch}')
        epoch_start_time = time.time()


        # Set current loss value
        current_loss = 0.0

        # Iterate over the DataLoader for training data
        train_loss = train(model)
        val_loss = evaluate(model)
        
        print('-' * 89)
        print('| end of epoch {:3d} | time: {:5.2f}s | valid loss {:5.2f} | train loss {:5.2f} |'.format(epoch, (time.time() - epoch_start_time),val_loss, train_loss))
        print('-' * 89)

#         if val_loss < best_val_loss:
#             best_val_loss = val_loss
#             best_model = model
#             torch.save(best_model, 'bestModel')

#         scheduler.step()
        val_losses.append(val_loss)
        train_losses.append(train_loss)
        epoch_averages[epoch -1].append(val_loss)
        if epoch % 5 == 0:
            plt.plot(train_losses, label = "Train_Loss")
            plt.plot(val_losses, label = "Val_Loss")
            plt.show()
            
    fold_scores.append(np.mean(np.array(val_losses)))
    scheduler.step()

    
#   # Print fold results
#   print(f'K-FOLD CROSS VALIDATION RESULTS FOR {k_folds} FOLDS')
    print('--------------------------------')
#   sum = 0.0
#   for key, value in results.items():
#     print(f'Fold {key}: {value} %')
#     sum += value
#   print(f'Average: {sum/len(results.items())} %')


print('*' * 89)
print('KFold has ended!!!!!!')
print(f'validation score is {np.mean(np.array(fold_scores))}')
print(f'rmse is {np.mean(np.array(fold_scores)) ** 0.5}')
print(len(fold_scores))



best_loss = float('inf')
best_epoch = 1
for epoch, values in enumerate(epoch_averages):
    print(epoch + 1)
    print(values)
    print(np.mean(values))
    if np.mean(values) < best_loss:
        best_loss = np.mean(values)
        best_epoch = epoch + 1
        
print(f'The best average loss is {best_loss} with epoch: {best_epoch}')

# Train Model

In [ ]:
train_loader = DataLoader(train_data, batch_size = batch_size, shuffle = True, num_workers = 4)


model.load_state_dict(torch.load('initialModel'))

for epoch in range(1, best_epoch + 1):
        print('-' * 89)
        print(f'Starting epoch {epoch}')
        epoch_start_time = time.time()
        train_loss = train(model)
        print(f'| end of epoch: {epoch} | time: {time.time() - epoch_start_time}s  | train loss: {train_loss} |')
        print('-' * 89)

In [ ]:
torch.save(model, 'bestModel')